In [1]:
import json as j #for parsing text data into json object
import pandas as pd #for dataframes
import re
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, chi2

In [10]:
from sklearn.pipeline import Pipeline

In [5]:
json_data = None
with open('/home/unnimaya/Downloads/yelp_academic_dataset_review.json') as data_file:
    lines = data_file.readlines()
    joined_lines = "[" + ",".join(lines) + "]" #brackets are added to make it a jason array
    json_data = j.loads(joined_lines) #If you have a JSON string, you can parse it by using the json.loads() method.

data = pd.DataFrame(json_data) # pandas daraframe is a container for dataset


In [6]:
stemmer = SnowballStemmer('english') #words to root
words = stopwords.words("english")

In [7]:
data['cleaned'] = data['text'].apply(lambda x: " ".join([stemmer.stem(i) for i in re.sub("[^a-zA-Z]", " ", x).split() if i not in words]).lower())

In [8]:
X_train,X_test,y_train,y_test = train_test_split(data['cleaned'],data.stars,test_size = 0.2)


In [11]:
pipeline = Pipeline([('vect', TfidfVectorizer(ngram_range=(1, 2), stop_words="english", sublinear_tf=True)),
                     ('chi',  SelectKBest(chi2, k=10000)),
                     ('clf', LinearSVC(C=1.0, penalty='l1', max_iter=3000, dual=False))])

In [12]:
model = pipeline.fit(X_train, y_train)

In [13]:
vectorizer = model.named_steps['vect']
chi = model.named_steps['chi']
clf = model.named_steps['clf']

In [14]:
feature_names = vectorizer.get_feature_names()
feature_names = [feature_names[i] for i in chi.get_support(indices=True)]
feature_names = np.asarray(feature_names)

In [15]:
target_names = ['1', '2', '3', '4', '5']
print("top 10 keywords per class:")
for i, label in enumerate(target_names):
    top10 = np.argsort(clf.coef_[i])[-10:]
    print("%s: %s" % (label, " ".join(feature_names[top10])))

top 10 keywords per class:
1: revolt stay far empti sign horribl singl star zero star nose dive place joke worst negat star
2: correct price gate like fun rate flavor weak sit steam left movi continu search nd star experienc better second star
3: decent need decent experi hate relationship ok best worst humm appreci effort just expect mix feel definit ok
4: lost star knock star reason star took star dock star fri enjoy thing star yes fan yay fan fifth star
5: custom life nami redseven ridicul good favorit bar amaz worth everi unmatch best stop read


In [18]:
print("accuracy score: " + str(model.score(X_test, y_test)))

accuracy score: 0.5818146231133922


In [17]:
print(model.predict(['that was an awesome place. Great food!']))

[5]
